# 01. Azure ML/Azure Data Factory Demo - Environment Setup
This notebook (designed to run from an Azure ML Compute Instance) connects to an AML workspace, and registers two tabular datasets from CSVs located within the `./sample_data` directory. These datasets (the Boston home prices and Iris Setosa datasets, respectively) are intended purely to be representative datasets that can be retrieved from the workspace and consumed within an AML pipeline. The intention of this demo is to create and execute an AML pipeline which accepts a single argument, and writes a file to an AML-linked blob store. This file can then be copied into an Azure SQL database for consumption in downstream reports. The final cells in this notebook create a table in an Azure SQL database (using pyodbc) that can serve as an effective sink for these data.

### Import required packages

In [ ]:
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute, DataFactoryCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData
from azureml.data.output_dataset_config import OutputTabularDatasetConfig, OutputDatasetConfig, OutputFileDatasetConfig
from azureml.data.datapath import DataPath
from azureml.data.data_reference import DataReference
from azureml.data.sql_data_reference import SqlDataReference
from azureml.pipeline.steps import DataTransferStep
import logging

### Connect to AML workspace, provision compute cluster for pipeline execution, and get references to default datastore and Key Vault

In [ ]:
#Connect to AML Workspace
ws = Workspace.from_config()

#Select AML Compute Cluster
cpu_cluster_name = 'cpucluster'

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found an existing cluster, using it instead.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D13_V2',
                                                           min_nodes=0,
                                                           max_nodes=1)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True)
    
#Create Data Factory Compute for DataTransferStep
def get_or_create_data_factory(workspace, factory_name):
    try:
        return DataFactoryCompute(workspace, factory_name)
    except ComputeTargetException as e:
        if 'ComputeTargetNotFound' in e.message:
            print('Data factory not found, creating...')
            provisioning_config = DataFactoryCompute.provisioning_configuration()
            data_factory = ComputeTarget.create(workspace, factory_name, provisioning_config)
            data_factory.wait_for_completion()
            return data_factory
        else:
            raise e
data_factory_name = 'adfcompute'           
data_factory_compute = get_or_create_data_factory(ws, data_factory_name)

#Get Default Datastore
default_ds = ws.get_default_datastore()

# Get Default Key Vault
default_kv = ws.get_default_keyvault()

### Create Datasets
Upload CSVs from the `./sample_data` directory (Boston Home Prices and Iris Setosa, respectively) and register as tabular datasets

In [ ]:
# Create two datasets in default storage account
default_ds.upload('./sample_data', target_path='sample_data', overwrite=True)

dataset_1 = Dataset.Tabular.from_delimited_files((default_ds, 'sample_data/a.csv'))
dataset_1.register(ws, 'Dataset_One', create_new_version=True)

dataset_2 = Dataset.Tabular.from_delimited_files((default_ds, 'sample_data/b.csv'))
dataset_2.register(ws, 'Dataset_Two', create_new_version=True)

### Set secrets in Azure Key Vault
The sample code below is configured to add Azure SQL credential details (server name, database name, username and password) to an Azure Key Vault for secure storage

In [ ]:
# default_kv.set_secret('SQL-SERVER', 'YOUR-SERVER-NAME')
# default_kv.set_secret('SQL-DATABASE', 'YOUR-DATABASE-NAME')
# default_kv.set_secret('SQL-USERNAME', 'YOUR-USERNAME')
# default_kv.set_secret('SQL-PASSWORD', 'YOUR-PASSWORD')

### Create Table in Azure SQL Database
Create a table inside an Azure SQL database that results will eventually be written to via Azure Data Factory

In [ ]:
import pyodbc 
server = default_kv.get_secret('SQL-SERVER')
database = default_kv.get_secret('SQL-DATABASE')
username = default_kv.get_secret('SQL-USERNAME')
password = default_kv.get_secret('SQL-PASSWORD') 
with pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password) as cnxn:
    cursor = cnxn.cursor()

    sql_command = """
    CREATE TABLE mydata (A float, B float, TIME datetime)
    """
    cursor.execute(sql_command)